In this step, we're going to generate our own glove based on our corpus. 

In [4]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer 
from gensim.utils import simple_preprocess
import spacy

In [56]:
import en_core_web_sm

nlp = en_core_web_sm.load(disable=['parser', 'ner'])

def lemmatization(sent, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
#    for sent in texts:
    doc = nlp(" ".join(sent)) 
    texts_out.extend([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [46]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

def remove_stopwords(doc):
    return [word for word in simple_preprocess(str(doc)) if word not in stop_words]


In [143]:
bdf = pd.read_csv("bDMARD without URL for Glove.csv")
cdf = pd.read_csv("cDMARD without URL for Glove.csv")

In [144]:
sentList = []

for tweet in cdf.tweet:
    sentList.append(lemmatization(remove_stopwords(tweet)))
for tweet in bdf.tweet:
    sentList.append(lemmatization(remove_stopwords(tweet)))

In [148]:
#importing the glove library
#credit to: https://medium.com/analytics-vidhya/word-vectorization-using-glove-76919685ee0b
from glove import Corpus, Glove
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(sentList, window=10)
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=300, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=40, no_threads=4, verbose=True)

glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 40 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39


In [7]:
from glove import  Glove

In [8]:
def load_glove_index(loc):
    f = open(loc,encoding="utf8")
    embeddings_index = {}
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

#creates word embedding matrix
def create_emb():
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in word_dict.items():
        if i >= vocab_size:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


In [149]:
import io
with io.open("myGlove300.txt",  "w", encoding="utf-8") as f:
    for key, val in glove.dictionary.items():
        f.write(key+ " "+ re.sub("(\\n|\[|\])", "",np.array2string(glove.word_vectors[val])) +"\n")